In [11]:
!pip install openai langgraph
!pip install langchain
!pip install langchain-community # install the langchain_community package

In [4]:

import openai
from langchain.chat_models import ChatOpenAI

# Set your OpenAI API key (this should be stored more securely, see note below)
openai.api_key = "Use Your Own OpenAI api Key"

# Pass the openai_api_key to the ChatOpenAI constructor
model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai.api_key)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00


<ipython-input-4-4eab7a0e00a4>:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai.api_key)


In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [6]:
config = {"configurable": {"thread_id": "abc123"}}

In [7]:
from langchain_core.messages import HumanMessage
query = "Hi! I'm Sohail."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hello Sohail! How can I assist you today?


In [10]:
# Keep asking questions until user says "bye" or "exit"
while query.lower() not in ["bye", "exit"]:
    query = input("You: ")
    input_messages = [HumanMessage(query)]
    output = app.invoke({"messages": input_messages}, config)
    output["messages"][-1].pretty_print()

You: Hello
================================== Ai Message ==================================

Hello! How can I assist you today?
You: I am sohail
================================== Ai Message ==================================

Nice to meet you, Sohail! How can I assist you today?
You: Do you know my name?
================================== Ai Message ==================================

Yes, your name is Sohail.
You: What was my last question
================================== Ai Message ==================================

Your last question was "Do you know my name?"
You: ok bye
================================== Ai Message ==================================

Goodbye! Have a great day, Sohail!
You: bye
================================== Ai Message ==================================

Goodbye! Take care!
